In [227]:
import xarray as xr
import geopandas as gpd
import pandas as pd
from dask.diagnostics import ProgressBar

# Read in the UTCI data

This is 3 hourly historical data, and will be used for training the mortality curves

In [5]:
dataset =  xr.open_mfdataset('/gws/pw/j05/cop26_hackathons/bristol/project10/utci_projections_1deg/HadGEM3-GC31-LL/historical/r1i1p1f3/*.nc')
dataset

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 180, lon: 360, time: 86400)
Coordinates:
  * time      (time) object 1985-01-01 03:00:00 ... 2015-01-01 00:00:00
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Dimensions without coordinates: bnds
Data variables:
    utci      (time, lat, lon) float64 dask.array<chunksize=(2880, 180, 360), meta=np.ndarray>
    lat_bnds  (time, lat, bnds) float64 dask.array<chunksize=(2880, 180, 2), meta=np.ndarray>
    lon_bnds  (time, lon, bnds) float64 dask.array<chunksize=(2880, 360, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

# Read in shapefiles for city locations

In [37]:
shps = gpd.read_file("repo/project10/data/raw_data/ne_10m_populated_places_simple/ne_10m_populated_places_simple.shp")
print(shps)

      scalerank  natscale  labelrank              featurecla  \
0            10         1          8         Admin-1 capital   
1            10         1          8         Admin-1 capital   
2            10         1          8         Admin-1 capital   
3            10         1          8         Admin-1 capital   
4            10         1          8         Admin-1 capital   
...         ...       ...        ...                     ...   
7338          0       600          1         Admin-1 capital   
7339          0       600          1         Admin-1 capital   
7340          0       600          3         Admin-1 capital   
7341          0       600          0         Admin-0 capital   
7342          0       600          0  Admin-0 region capital   

                        name namepar              namealt  diffascii  \
0     Colonia del Sacramento    None                 None          0   
1                   Trinidad    None                 None          0   
2              

In [33]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

download_and_unzip("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_populated_places.zip")
shp = gpd.read_file("ne_10m_populated_places.shp")
print(shp)

      SCALERANK  NATSCALE  LABELRANK              FEATURECLA  \
0            10         1          8         Admin-1 capital   
1            10         1          8         Admin-1 capital   
2            10         1          8         Admin-1 capital   
3            10         1          8         Admin-1 capital   
4            10         1          8         Admin-1 capital   
...         ...       ...        ...                     ...   
7338          8        10          8         Admin-1 capital   
7339          8        10          8  Meteorological Station   
7340          8        10          8         Populated place   
7341          8        10          8         Populated place   
7342          8        10          8         Populated place   

                        NAME NAMEPAR NAMEALT  DIFFASCII  \
0     Colonia del Sacramento    None    None          0   
1                   Trinidad    None    None          0   
2                Fray Bentos    None    None          

In [39]:
shps.columns

Index(['scalerank', 'natscale', 'labelrank', 'featurecla', 'name', 'namepar',
       'namealt', 'diffascii', 'nameascii', 'adm0cap', 'capalt', 'capin',
       'worldcity', 'megacity', 'sov0name', 'sov_a3', 'adm0name', 'adm0_a3',
       'adm1name', 'iso_a2', 'note', 'latitude', 'longitude', 'changed',
       'namediff', 'diffnote', 'pop_max', 'pop_min', 'pop_other', 'rank_max',
       'rank_min', 'geonameid', 'meganame', 'ls_name', 'ls_match', 'checkme',
       'min_zoom', 'ne_id', 'geometry'],
      dtype='object')

In [45]:
(shps.query('name in ["Alofi", "Florida"]'))

,scalerank,natscale,labelrank,featurecla,name,namepar,namealt,diffascii,nameascii,adm0cap,...,rank_max,rank_min,geonameid,meganame,ls_name,ls_match,checkme,min_zoom,ne_id,geometry
4,10,1,8,Admin-1 capital,Florida,None,None,0,Florida,0.0,...,7,7,3442585.0,None,None,0,0,7.0,1159112703,POINT (-56.21500 -34.09900)
1812,8,10,8,Admin-1 capital,Alofi,None,None,0,Alofi,0.0,...,0,0,4036284.0,None,None,0,0,7.0,1159151741,POINT (-169.91363 -19.06599)


In [51]:
shps.groupby(['adm0name']).agg('size')

adm0name
Afghanistan       33
Aland              1
Albania           26
Algeria           51
American Samoa     1
                  ..
Vietnam           60
Western Sahara     1
Yemen             20
Zambia            34
Zimbabwe          20
Length: 228, dtype: int64

In [74]:
populated = (shps
    .assign(
        lon=shps['geometry'].x,
        lat=shps['geometry'].y,
        pop=(shps['pop_max']+shps['pop_min'])/2)
    .filter(['name', 'adm0name', 'pop', 'lon', 'lat']))
populated

,name,adm0name,pop,lon,lat
0,Colonia del Sacramento,Uruguay,21714.0,-57.840002,-34.479999
1,Trinidad,Uruguay,21093.0,-56.900997,-33.543999
2,Fray Bentos,Uruguay,23279.0,-58.303997,-33.138999
3,Canelones,Uruguay,19698.0,-56.284001,-34.538004
4,Florida,Uruguay,32234.0,-56.214998,-34.099002
...,...,...,...,...,...
7338,Rio de Janeiro,Brazil,6879087.5,-43.226967,-22.923077
7339,São Paulo,Brazil,14433147.5,-46.626966,-23.556734
7340,Sydney,Australia,4135711.0,151.183234,-33.918065
7341,Singapore,Singapore,4236614.5,103.853875,1.294979


populated.to_parquet('../data/processed_data/populated.parquet.gz', compression='gzip')

In [229]:
france = populated.query('adm0name == "France"').reset_index()
france.to_parquet('../data/processed_data/france.parquet.gz', compression='gzip')

<ipython-input-229-0662050d4dd4>:2: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  france.to_parquet('../data/processed_data/france.parquet.gz', compression='gzip')


In [1]:
import xarray as xr
import geopandas as gpd

# Read in the UTCI data

This is 3 hourly historical data, and will be used for training the mortality curves

In [104]:
utci =  xr.open_mfdataset('/gws/pw/j05/cop26_hackathons/bristol/project10/utci_projections_1deg/HadGEM3-GC31-LL/historical/r1i1p1f3/*.nc', parallel=True)
utci

<xarray.Dataset>
Dimensions:   (bnds: 2, lat: 180, lon: 360, time: 86400)
Coordinates:
  * time      (time) object 1985-01-01 03:00:00 ... 2015-01-01 00:00:00
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
Dimensions without coordinates: bnds
Data variables:
    utci      (time, lat, lon) float64 dask.array<chunksize=(2880, 180, 360), meta=np.ndarray>
    lat_bnds  (time, lat, bnds) float64 dask.array<chunksize=(2880, 180, 2), meta=np.ndarray>
    lon_bnds  (time, lon, bnds) float64 dask.array<chunksize=(2880, 360, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7

Subset to only have UK locations

In [210]:
# For the xarray object
# - select the coordinates for a given city
# - calculate the monthly mean UTCI at those coordinates
# - return a dataframe with utci over time, as well as the city info
def mean_utci_position(utci, pop, row):
    with ProgressBar():
        out = (utci
               .sel(lon=pop['lon'][row], lat=pop['lat'][row], method='nearest')['utci']
               .resample(time="1M")
               .mean()
               .compute()
               .to_dataframe()
               .assign(
                    name=pop['name'][row],
                    adm0name=pop['adm0name'][row],
                    pop=pop['pop'][row],
                    lon_orig=pop['lon'][row],
                    lat_orig=pop['lat'][row]
                ))
    return(out)

In [211]:
m = mean_utci_position(utci, populated, 1)
m

[                                        ] | 0% Completed |  1min 36.9s


KeyboardInterrupt: 

In [218]:
def country_utci(utci, populated, country):
    sub = populated.query('adm0name == "{}"'.format(country)).reset_index()
    print("Calculating " + str(sub.shape[0]) + " regions")
    out = [mean_utci_position(utci, sub, row) for row in range(sub.shape[0])]
    return(pd.concat(out))

In [223]:
out = country_utci(utci, populated, "Monaco")

Calculating 1 regions
[########################################] | 100% Completed | 12min 33.9s


In [224]:
out

,lat,lon,utci,name,adm0name,pop,lon_orig,lat_orig
time,,,,,,,,
1985-01-30 00:00:00,43.5,7.5,273.848787,Monaco,Monaco,36371.0,7.406913,43.739646
1985-02-30 00:00:00,43.5,7.5,282.069428,Monaco,Monaco,36371.0,7.406913,43.739646
1985-03-30 00:00:00,43.5,7.5,281.880166,Monaco,Monaco,36371.0,7.406913,43.739646
1985-04-30 00:00:00,43.5,7.5,285.968089,Monaco,Monaco,36371.0,7.406913,43.739646
1985-05-30 00:00:00,43.5,7.5,292.263468,Monaco,Monaco,36371.0,7.406913,43.739646
...,...,...,...,...,...,...,...,...
2014-09-30 00:00:00,43.5,7.5,295.759083,Monaco,Monaco,36371.0,7.406913,43.739646
2014-10-30 00:00:00,43.5,7.5,290.545414,Monaco,Monaco,36371.0,7.406913,43.739646
2014-11-30 00:00:00,43.5,7.5,281.987321,Monaco,Monaco,36371.0,7.406913,43.739646


In [230]:
france = country_utci(utci, populated, "France")

Calculating 71 regions
[########################################] | 100% Completed |  6min 20.9s
[########################################] | 100% Completed |  2min 31.7s
[########################################] | 100% Completed | 11min 55.1s
[########################################] | 100% Completed |  3min 22.1s
[########################################] | 100% Completed |  5min 32.8s
[########################################] | 100% Completed | 26.2s
[########################################] | 100% Completed |  8.4s
[########################################] | 100% Completed |  1min 30.8s
[########################################] | 100% Completed |  3.0s
[########################################] | 100% Completed |  4min 41.1s
[########################################] | 100% Completed |  1min 31.3s
[########################################] | 100% Completed | 31.0s
[########################################] | 100% Completed | 28.3s
[########################################] | 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[########################################] | 100% Completed |  7min 21.6s
[                                        ] | 0% Completed | 15.5s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[########################################] | 100% Completed |  8min 32.5s
[                                        ] | 0% Completed |  1.0s

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[########################################] | 100% Completed |  4min  6.9s
[########################################] | 100% Completed |  2.9s
[########################################] | 100% Completed |  3min 14.9s
[########################################] | 100% Completed | 19.2s


In [1]:
france

NameError: name 'france' is not defined

In [ ]:
type(france)